# TIES483 Assignment, Mikael Myyrä

***

## Problem 1

A window is being built and the bottom is a rectangle and the top is a semicircle. If there is 12 m of framing materials what must the dimensions of the window be to make the window area as big as possible?

Model the decision problem as an optimization problem and solve it with a method of your choosing. **Analyse the result!**

***

The relevant measurements (circumference and area) of the window can be expressed as functions of width $w$ and height $h$. The circumference is $w + 2h$ for the rectangular part and $\pi \frac{w}{2}$ for the circular part, so the total circumference is $C(w, h) = (1 + \frac{\pi}{2})w + 2h$. The area is $wh$ for the rectangular part and $\pi (\frac{w}{2})^2$ for the circular part, so the total area is $A(w, h) = wh + \frac{\pi}{4} w^2$. We're trying to maximize $A$ (and thus minimize $-A$) subject to the linear constraint $C = 12$. Also, $w$ and $h$ must be greater than zero because this is a real window. In formal terms, the problem is

$$
\begin{align}
\min \qquad & -wh - \frac{\pi}{4} w^2 \\
\text{s.t.} \qquad & (1 + \frac{\pi}{2})w + 2h = 12 \\
& w, r > 0
\end{align}
$$

In this simple case it should be possible to find a solution analytically using the stationarity rule from the KKT conditions. Let's try it. The Lagrangian for this problem is

$$
L(w, h) = -wh - \frac{\pi}{4} w^2 - \lambda(1 + \frac{\pi}{2})w - 2\lambda h
$$

and its gradient

$$
\nabla L(w, h) = (-h - \frac{\pi}{2} w - \lambda(1 + \frac{\pi}{2}), -w - 2\lambda).
$$

We're looking for points where
$$
\nabla L(w, h) = \mathbf{0} \\
\begin{cases}
-h - \frac{\pi}{2}w - \lambda(1 + \frac{\pi}{2}) = 0 \\
-w - 2\lambda = 0
\end{cases} \\
w = -\frac{\lambda}{2} \\
-h + (\frac{\pi}{4} - (1 + \frac{\pi}{2}))\lambda = 0 \iff h = (-\frac{\pi}{4} - 1)\lambda
$$

Applying the constraint, we find feasible values for $\lambda$:

$$
\begin{align}
(1 + \frac{\pi}{2})w + 2h &= 12 \\
(1 + \frac{\pi}{2})(-\frac{\lambda}{2}) + 2(-\frac{\pi}{4} - 1)\lambda &= 12 \\x
(-\frac12 - \frac{\pi}{4} - \frac{\pi}{2} - 2)\lambda &= 12 \\
(-\frac{3\pi}{4} - \frac{5}{2})\lambda &= 12 \\
\lambda &= \frac{-12}{\frac{3\pi}{4} + \frac{5}{2}}
\end{align}
$$

now, applying $\lambda$ back to $w$ and $h$, we get

$$
w = -\frac{\lambda}{2} = \frac{12}{\frac{3\pi}{2} + 5} \approx 1.236 \\
h = (-\frac{\pi}{4} - 1) \lambda 
  = \frac{3\pi + 12}{\frac{3\pi}{4} + \frac{5}{2}} \approx 4.412.
$$

Analysis: Because we used the KKT conditions to come up with this solution analytically, we already know it's optimal. Just to check that the calculations were correct, you can plug $w$ and $h$ back into the constraint function to ensure it holds. Plugging said values into the objective function and flipping the sign gives the optimal area of the window, which comes out to be approximately $6.653 m^2$.

***

## Problem 2

The 10-dimensional Robsenbrock function (one of the variants) is defined as
$$
f(\mathbf{x}) = \sum_{i=1}^{9} 100 (x_{i+1} - x_i^2 )^2 + (1-x_i)^2
$$
for $x\in\mathbb R^{10}$. 

Compare at least two different optimization method's performance in minimizing this function over $\mathbb R^{10}$. You can decide the method of comparison as the one that makes most sense to you. **Analyze the results!**

***

***

## Problem 3

The task is to solve a black-box optimization problem where the objective and constraint function values can be obtained by calling an executable. The executable <i>prob3</i> will be available at the course website http://users.jyu.fi/~jhaka/opt/ along with instructions on how to use it in the <i>README</i> file. 

The format of the problem is
$$
\begin{align}
\min \ &f(x)\\
\text{s.t. }&h_1(x) = 0\\
        &h_2(x) = 0\\
        &g_1(x) \geq 0\\
        &g_2(x) \geq 0\\
        &g_3(x) \geq 0\\
        &g_4(x) \geq 0\\
        &x\in \mathbb R^4.        
\end{align}
$$
Solve the optimization problem by using the tools and optimization method of your choosing. **Analyse the results!**

***

***

## Problem 4

Study biobjective optimization problem
$$
\begin{align}
\min \ &(\|x-(1,0)\|,\|x-(0,1)\|)\\
\text{s.t. }&x\in \mathbb R^2.
\end{align}
$$
Try to generate an evenly spread representation of the Pareto front. Plot the results in both the decision and objective spaces. **Analyze the results!**

***